In [ ]:
# Testing Cell

from aviary.utils.doctape import glue_variable, get_variable_name
from aviary.api import Aircraft, AviaryProblem
glue_variable('capi',AviaryProblem.check_and_preprocess_inputs.__name__,md_code=True)

glue_variable(get_variable_name(Aircraft.CrewPayload.Design.NUM_PASSENGERS), md_code=True)
glue_variable(get_variable_name(Aircraft.CrewPayload.NUM_PASSENGERS), md_code=True)
glue_variable(get_variable_name(Aircraft.CrewPayload.NUM_BUSINESS_CLASS), md_code=True)
glue_variable(get_variable_name(Aircraft.CrewPayload.NUM_FIRST_CLASS), md_code=True)
glue_variable(get_variable_name(Aircraft.CrewPayload.NUM_TOURIST_CLASS), md_code=True)


# Multi-Mission Optimization

## Overview

Multi-missions adds the capability to optimize an aircraft design considering two or more missions that the aircraft will perform. This is in contrast to designing an aircraft for a single mission, and hopefully helps to represent the conditions the aircraft will be flying in better. In the example provided, a large single aisle passenger aircraft is designed based on a mission with every seat filled, and a second 'deadhead' mission where there are no passengers. A weighting is provided by the user to determine how often the missions are full of passengers vs. how often the flights are empty. A cargo example could be constructed where one mission is designed for maximum range and a second mission designed for maximum payload. 

To impact the structure of aviaryproblems as little as possible, after instantiation of the pre-mission, mission, and post-mission systems, the connections between those systems are created. Then those groups are then copied over into a regular openmdao problem. This enables the use all the basic aviary connection and checking functions with minimal modification. 

The objective function is based on combining the fuel-burn values from both missions and the optimizers objective is to minimize the weighted fuel-burn. Other objectives, like max range, have not been tested yet.

## Design vs. As-Flown
To support the need to design an aircraft with a certain number of seats, but then possibly fly missions with less passengers, a distinction in the metadata was introduced between {glue:md}`Aircraft.CrewPayload.Design.NUM_PASSENGERS` and {glue:md}`Aircraft.CrewPayload.NUM_PASSENGERS`. The individual passenger classes ({glue:md}`Aircraft.CrewPayload.NUM_FIRST_CLASS`, {glue:md}`Aircraft.CrewPayload.NUM_BUSINESS_CLASS`, {glue:md}`Aircraft.CrewPayload.NUM_TOURIST_CLASS`) also have these distinctions. The Design values represent how many seats are available in the aircraft. Whereas the non-design values represent an as-flow value of how many passengers are on a particular flight. 

A number of checks exist in {glue:md}`capi` to help the user in the case that incomplete as-flow or design passenger information is provided. Please review the [Multi-Mission Theory](../examples/multi_mission) for further details.

## Example
An example of a multi-mission as well as Setup, Theory, and Results, is presented in [Multi-Mission Examples](../examples/multi_mission).